In [6]:
import os
import sys

import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import colors
from hydra import initialize, compose
import pickle
from glob import glob
from pprint import pprint
from tqdm import tqdm
import galsim
from copy import deepcopy

# set paths to various directories based on the machine this code is being executed on
try:
    with initialize(version_base=None, config_path='config'):  
        config = compose(config_name='config.yaml')  # overrides=['machine=uzay']
except:
    with initialize(version_base=None, config_path='../../config'):  
        config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir  = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.lenses.test import SampleStrongLens
from mejiro.plots import diagnostic_plot, plot, plot_util, overplot
from mejiro.analysis import stats
from mejiro.utils import util
from mejiro.helpers import gs
from mejiro.instruments.roman import Roman

In [36]:
from astropy.table import Table

# Path to your ECSV file
file_path = '/grad/bwedig/mejiro/mejiro/data/Roman_zeropoints_20240301.ecsv'

# Read the ECSV file
table = Table.read(file_path, format="ascii.ecsv")

# Print the table to see its contents
table.show_in_notebook()

idx,detector,element,VegaMag,ABMag,STMag,FLAM,FNU,PHOTLAM,PHOTNU
0,WFI01,F062,26.417663323428016,26.57551933086481,26.880632649382186,6.423133376578637e-20,8.507310180127394e-31,2.0375262991355374e-08,2.6986623522593194e-19
1,WFI01,F087,25.626924239161607,26.22624275057486,27.242258374678336,4.603590128457495e-20,1.1735535182435983e-30,2.0259717908486147e-08,5.16463511448471e-19
2,WFI01,F106,25.64881610600677,26.315214476595152,27.76074267421951,2.8556365391332294e-20,1.0812203456695343e-30,1.5315888250549722e-08,5.799004796850561e-19
3,WFI01,F129,25.290631156180385,26.319402188403885,28.174524763703083,1.9506982507197113e-20,1.0770580837770768e-30,1.2634213997778904e-08,6.9758520127122e-19
4,WFI01,F158,24.99266535652234,26.32251053616207,28.628585520732255,1.2840022705327068e-20,1.0739789902225378e-30,1.0235625974669392e-08,8.561392375116519e-19
5,WFI01,F184,24.292674328339736,25.85460209959829,28.490403011052088,1.4582728680228105e-20,1.652567319829496e-30,1.3531084213521828e-08,1.5333911823680173e-18
6,WFI01,F146,26.374869539414142,27.538551048094657,29.631552071849306,5.097757488221362e-21,3.5041249252149677e-31,3.68394379042415e-09,2.532289793883505e-19
7,WFI01,F213,23.97336286090499,25.814939003626883,28.771874732696283,1.1252528120267351e-20,1.7140535998819194e-30,1.2105169413721794e-08,1.8439331134306342e-18
8,WFI02,F062,26.456551628991388,26.606875772410554,26.9047246288177,6.282176710902194e-20,8.26513004137086e-31,1.986156833584717e-08,2.613082259154974e-19
9,WFI02,F087,25.69202112174527,26.298118455467243,27.31432397926335,4.307947562362592e-20,1.098380003596168e-30,1.8960296833138415e-08,4.834230361974432e-19


In [37]:
zp_dict = {}

for i in range(1, 19):
    zp_dict[f'SCA{str(i).zfill(2)}'] = {}

pprint(zp_dict)

{'SCA01': {},
 'SCA02': {},
 'SCA03': {},
 'SCA04': {},
 'SCA05': {},
 'SCA06': {},
 'SCA07': {},
 'SCA08': {},
 'SCA09': {},
 'SCA10': {},
 'SCA11': {},
 'SCA12': {},
 'SCA13': {},
 'SCA14': {},
 'SCA15': {},
 'SCA16': {},
 'SCA17': {},
 'SCA18': {}}


In [38]:
for row in table:
    sca_id = int(row['detector'][3:])
    band = row['element']
    zp = row['ABMag']
    zp_dict[f'SCA{str(sca_id).zfill(2)}'][band] = zp

pprint(zp_dict)

{'SCA01': {'F062': 26.57551933086481,
           'F087': 26.22624275057486,
           'F106': 26.315214476595152,
           'F129': 26.319402188403885,
           'F146': 27.538551048094657,
           'F158': 26.32251053616207,
           'F184': 25.85460209959829,
           'F213': 25.814939003626883},
 'SCA02': {'F062': 26.606875772410554,
           'F087': 26.298118455467243,
           'F106': 26.35167233614996,
           'F129': 26.33861065284702,
           'F146': 27.57759502002258,
           'F158': 26.369360993802363,
           'F184': 25.912884859107987,
           'F213': 25.857202340635656},
 'SCA03': {'F062': 26.601314862505767,
           'F087': 26.28313679639355,
           'F106': 26.336674952482202,
           'F129': 26.33438763453907,
           'F146': 27.56623083572626,
           'F158': 26.359307071686374,
           'F184': 25.910783654610498,
           'F213': 25.854265987972553},
 'SCA04': {'F062': 26.541296980333765,
           'F087': 26.2702898970